# Installation

In [ ]:
%pip install --quiet --upgrade diffusers transformers accelerate mediapy
!pip install openai
!pip install Image
!pip install optimum
!pip install optimum.intel
!pip install openvino

#Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##Imports

In [ ]:
from openai import OpenAI
import mediapy as media
import random
import sys
import torch
import json
import textwrap
from diffusers import AutoPipelineForText2Image

import cv2
from PIL import Image
import numpy as np

#Prompt preparation

In [ ]:
client = OpenAI(api_key="sk-proj-JYHoORcm3bg4YuzFKDNsT3BlbkFJTCr1mK3WwSSlzkPJ1eMV")

# Story context
current_context = '''
Gasping, eyes red and glaring into the dimness of her bedroom, Elanna sits bolt upright in bed. Or tries to; her legs and left arm are tangled in the sheets, immobilized, her grey tank top gone black and clinging between her breasts with terrorsweat. Her right hand is clamped tight around her Glock 23, which always lives on her bedside table. Only the fact that it’s also always safed has kept her from punching a good-sized hole in her apartment wall, maybe in a neighbor as well.
That'd make a great headline. “Nightmare Cop Shoots Kid Next Door, Claims Monster in her Dream Made her Do It.”
Shit.
Draping a robe around herself- the place is way too chilly for t and scanties- she fumbles her way to the kitchen to douse her face. Third time in a month she’s had that dream. She wonders what the Department headtwister would think of it.
If she dared tell him.
Instead, she turns on the idiot box, just to have a human voice around, and stares out at the snow from the fifth floor. It looks so clean from up there. Everything always does.
'''

#will be fed to GPT
character_name= ["Elanna Porter (Eraille)"]

# Opening Humaira .json
f = open("/content/drive/MyDrive/GENAI/place_and_scene_data.json")
my_file = json.load(f)
print(my_file[0])
# will be fed to CLIP
event_description = f"{my_file[0]['scene_location_name']}: {my_file[0]['scene_location_description']}"

first_para_disc = my_file[0]['scene_entry_content']
para_no = 1 # will chanfge once we add the BERT thingie
prompt = f'''
You are supposed to generate paragraph number#{para_no} of a sotry (no more than 70 words). Consider that the first paragraph's desciption is:{first_para_disc}. The characters involved are: {character_name}. Write a creative, thrilling, and catchy paragraph (70 words only)
'''
prompt = textwrap.fill(prompt, width=100)
delimeter = ","
# print(f"\n place name: {my_file[0]['scene_location_name']}")
print(f"\n Event description: {my_file[0]['scene_location_name']}: {my_file[0]['scene_location_description']}")
print(f"\n Characters involved: {delimeter.join(character_name)}")
print(f"\n current context: {current_context}")

In [ ]:
# PROMPT TO GPT
print(f"Prompt to GPT: {prompt}")

In [ ]:
#PROMPT TO SD
print(f"PROMPT TO SD: {textwrap.fill(event_description, width=100)}")

##Testing GPT (will deduct $$$ on each run)

In [ ]:

my = client.chat.completions.create(
    messages=[
        {"role": "user", "content": prompt}
    ],
    model = "gpt-3.5-turbo-0125"
)
gpt_answer = my.choices[0].message.content
print(textwrap.fill(gpt_answer, width=100))

# Testing Stable Diffusion

In [ ]:
pipe = AutoPipelineForText2Image.from_pretrained(
    "stabilityai/sdxl-turbo",
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
    )

In [ ]:
pipe = pipe.to("cuda")
seed = random.randint(0, sys.maxsize)

num_inference_steps = 4

images = pipe(
    prompt = gpt_answer+event_description,
    guidance_scale = 0.0,
    num_inference_steps = num_inference_steps,
    generator = torch.Generator("cuda").manual_seed(seed),
    ).images

print(textwrap.fill(gpt_answer, width=100))
media.show_images(images)

In [ ]:
images[0].save("/content/Untitled/image.png")

## EXPERIMENTS

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id)

In [ ]:

prompt = '''Elanna Porter (Eraille) stood at the edge of the park, her breath forming wispy clouds in the frigid
air. The moon cast a luminous glow over the snow-covered landscape, illuminating her determined
expression. As she scanned the deserted streets, a sense of anticipation filled her. Tonight, she
would embark on a mission that would defy the odds and challenge her to confront the darkest corners
of her soul. The fate of the city lay in her hands.'''
image = pipe(prompt).images[0]

image.save("astronaut_rides_horse.png")

In [ ]:
dir = "/content/save"
# pipe.save_pretrained(dir)
loaded_pipe = StableDiffusionPipeline.from_pretrained(dir)
prompt = "At the stroke of three, silence descends upon the city, broken only by the faint echo of distant footsteps. Elanna Porter, her breath forming icy clouds in the frigid air, navigates the deserted streets. Her senses alert, she moves like a shadow, unseen yet vigilant. As the city sleeps under its snowy shroud, she is its silent guardian, ever watchful for the flicker of danger amidst the sparkling lights of dreams."
image = pipe(prompt).images[0]

# image.save("astronaut_rides_horse.png")

In [ ]:
from optimum.intel.openvino import OVStableDiffusionPipeline
model_id = "runwayml/stable-diffusion-v1-5"
ov_pipe = OVStableDiffusionPipeline.from_pretrained(model_id, export=True)
image = ov_pipe(prompt).images[0]